In [ ]:
import numpy as np

A = np.zeros([8, 8])

I = [0, 0, 0, 0, 1, 1, 1, 4, 1, 1, 7, 0, 2, 2, 2, 6, 1, 6]
J = [1, 6, 5, 3, 3, 6, 7, 6, 6, 0, 6, 7, 3, 2, 3, 0, 3, 4]

A[I, J] = 1
A

In [ ]:
F1 = np.insert(A.sum(1).cumsum(), 0, 0)
F2 = np.insert(A.sum(0).cumsum(), 0, 0)
F = (F1 + F2) / 2
F1, F2, F

In [ ]:
def part(A, p, q):
    As = []
    for i in range(len(p) - 1):
        As.append([])
        for j in range(len(q) - 1):
             As[-1].append(A[p[i]: p[i + 1], q[j]: q[j + 1]])
    return As

def load(A, p, q):
    As = part(A, p, q)
    L = np.zeros([len(p) - 1, len(q) - 1])
    for i, row in enumerate(As):
        for j, M in enumerate(row):
            L[i, j] = M.sum()
    return L

def pi_to_p(a, pi):
    return np.searchsorted(a, pi, 'right') - 1

def p_to_pi(a, p):
    return a[p]

op = [0, 2, 4, 8]
p = op
L = load(A, p, p)
pi1 = p_to_pi(F1, p)
pi2 = p_to_pi(F2, p)
pi = p_to_pi(F, p)
pi1, pi2, pi, L

In [ ]:
L = load(A, p, p)
r1 = L.max(1)
r2 = L.max(0)
r = np.maximum(r1, r2)
r1, r2, r

In [ ]:
def g(r):
    r = np.cumsum(r)
    r -= (1 + np.arange(r.shape[0])) / r.shape[0] * r[-1]
    return np.insert(r, 0, 0)

eta = 2
pi1 -= eta * g(r1)
pi2 -= eta * g(r2)
pi -= eta * g(r)

p1 = pi_to_p(F1, pi1)
p2 = pi_to_p(F2, pi2)
p = pi_to_p(F, pi)

pi1, F1, pi2, F2, pi, F, p1, p2, p, L, load(A, p1, p2), load(A, p, p), g(r1), g(r2), g(r)

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt

def vis(A, p, q, suf=''):
    As = part(A, p, q)

    fig, axss = plt.subplots(len(p) - 1, len(q) - 1, figsize=(20, 20), gridspec_kw={
                           'width_ratios': [q[j + 1] - q[j] for j in range(len(q) - 1)],
                           'height_ratios': [p[i + 1] - p[i] for i in range(len(p) - 1)]})

    fig.tight_layout(pad=0)
    fig.subplots_adjust(hspace=.1, wspace=.1)
    
    for i, (axs, row) in enumerate(zip(axss, As)):
        for j, (ax, M) in enumerate(zip(axs, row)):
            ax.matshow(M, extent=(q[j], q[j + 1], p[i + 1], p[i]), cmap=plt.cm.Blues, norm=mpl.colors.Normalize(vmin=0, vmax=1))
            w = q[j + 1] - q[j]
            h = p[i + 1] - p[i]
            ax.grid(color='black', linestyle='-', linewidth=3)
            ax.set_xticks(np.arange(q[j], q[j + 1] + 1, 1))
            ax.set_yticks(np.arange(p[i], p[i + 1] + 1, 1))
            if j == 0:
                ax.set_yticklabels(np.arange(p[i], p[i + 1] + 1, 1))
            else:
                ax.set_yticklabels([])
            if i == len(p) - 2:
                ax.set_xticklabels(np.arange(q[j], q[j + 1] + 1, 1))
                ax.xaxis.tick_bottom()
            else:
                ax.set_xticklabels([])

    # fig.savefig('fig{}.pdf'.format(suf))
    
    return fig

fig1 = vis(A, op, op)
fig2 = vis(A, p, p, 'SRPP')
fig3 = vis(A, p1, p2, 'RPP')

In [ ]:
loads = []
errs = []
iters = []
with open('../results/twitter7-rect-16.txt') as f:
    for line in f:
        if len(line.split()) != 4:
            continue
        iters += [int(line.split()[0])]
        loads += [int(line.split()[1])]
        errs += [float(line.split()[2])]

print(list(zip(loads, errs)))


In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

import numpy as np
import matplotlib.pyplot as plt

TDAcolors=['#1f77b4', '#ff7f0e', '#123a12', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
plt.rcParams['axes.prop_cycle'] = mpl.rcsetup.cycler(color=TDAcolors)
markers = ['o', '^', 's', '+', 'x', 'd', '*', '>', '.']

def make_patch_spines_invisible(ax):
    ax.set_frame_on(True)
    ax.patch.set_visible(False)
    for sp in ax.spines.values():
        sp.set_visible(False)

def plot1():
    fig, axs = plt.subplots(1, 1, sharex=True)
    ax = axs
    p1, = ax.plot(iters, np.array(loads) / 1000000, color=TDAcolors[0], label='Maximum load')
    ax.set_ylim(bottom=0)
    ax.set_title("Maximum load over iterations")
    ax.grid()
    sax = ax.twinx()
    sax.set_ylim(bottom=0)
    sax.set_ylabel('local optimality measure')
    # rax = ax.twinx()
    # rax.spines["right"].set_position(("axes", 1.2))
    # make_patch_spines_invisible(rax)
    # rax.spines["right"].set_visible(True)
    p2, = sax.plot(iters, errs, color=TDAcolors[1], label='Local optimality measure')
    # rax.set_ylim(bottom=0)
    # rax.set_ylabel('Local optimality measure')
    ax.legend([p1, p2], [p1.get_label(), p2.get_label()])
    ax.set_ylabel('Maximum load x 10^7')

    fig.tight_layout(pad=2.0)
    fig.savefig("../ResearchArtifacts/IPDPS/figures/twitter7-rect.pdf")

plot1()